In [1]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

In [32]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

In [33]:
trainset = torchvision.datasets.MNIST(root='./mnist', train=True,
                                        download=True, transform = transform)
testset = torchvision.datasets.MNIST(root='./mnist', train = False, download = True, transform = transform)

In [34]:
trainloader = torch.utils.data.DataLoader(dataset=trainset, batch_size=4, shuffle=True)
testloader = torch.utils.data.DataLoader(dataset=testset, batch_size=4, shuffle=False)

In [108]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 3, padding=1)
        self.conv2 = nn.Conv2d(20, 50, 3, padding= 1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(50*7*7, 500)
        self.fc2 = nn.Linear(500, 10)
    
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 50*7*7)
        x = F.relu(self.fc1(x))
#         x = F.relu(self.fc2(x))
        x = self.fc2(x)
        return x
    
net = LeNet()
net = net.cuda()

In [109]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr= 0.01, momentum = 0.9)

In [110]:
for epoch in range(3):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data

        # wrap them in Variable
        inputs, labels = Variable(inputs.cuda()), Variable(labels.cuda())
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs.cuda(), labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.data[0]
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

[1,  2000] loss: 0.360
[1,  4000] loss: 0.154
[1,  6000] loss: 0.113
[1,  8000] loss: 0.146
[1, 10000] loss: 0.110
[1, 12000] loss: 0.103
[1, 14000] loss: 0.107
[2,  2000] loss: 0.094
[2,  4000] loss: 0.081
[2,  6000] loss: 0.110
[2,  8000] loss: 0.088
[2, 10000] loss: 0.090
[2, 12000] loss: 0.107
[2, 14000] loss: 0.122
[3,  2000] loss: 0.086
[3,  4000] loss: 0.104
[3,  6000] loss: 0.082
[3,  8000] loss: 0.074
[3, 10000] loss: 0.093
[3, 12000] loss: 0.067
[3, 14000] loss: 0.100
Finished Training


In [111]:
correct = 0
total = 0
for data in testloader:
    images, labels = data
    outputs = net(Variable(images.cuda()))
    _, predicted = torch.max(outputs.data.cuda(), 1)
    total += labels.size(0)
    correct += (predicted == labels.cuda()).sum()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 98 %
